In [1]:
# Load the data from the Apporto machine to the Colab environment

from google.colab import files
uploaded = files.upload()

Saving HW4_FlightDelays.csv to HW4_FlightDelays.csv


In [2]:
import pandas as pd

df = pd.read_csv('HW4_FlightDelays.csv')

In [4]:
print(df.shape)
print(df.columns)

(1319, 7)
Index(['Binned_CRS_DEP_TIME', 'CARRIER', 'DEST', 'ORIGIN', 'Weather',
       'DAY_WEEK', 'Flight Status'],
      dtype='object')


In [5]:
#Looking for Nulls
print(df.isnull().sum())

Binned_CRS_DEP_TIME    0
CARRIER                0
DEST                   0
ORIGIN                 0
Weather                0
DAY_WEEK               0
Flight Status          0
dtype: int64


In [14]:
# Part: Variable Transformation
# Setting the datatype for the variables in the cvar_list to be categorical
cvar_list = ['Binned_CRS_DEP_TIME', 'CARRIER', 'DEST', 'ORIGIN', 'Weather', 'DAY_WEEK', 'Flight Status']

In [15]:
# Converting the categorical variables into dummies
df1 = df.copy()
df1 = pd.get_dummies(df, prefix_sep='_')

list(df1.columns)

['Binned_CRS_DEP_TIME',
 'Weather',
 'DAY_WEEK',
 'CARRIER_CO',
 'CARRIER_DH',
 'CARRIER_DL',
 'CARRIER_MQ',
 'CARRIER_OH',
 'CARRIER_RU',
 'CARRIER_UA',
 'CARRIER_US',
 'DEST_EWR',
 'DEST_JFK',
 'DEST_LGA',
 'ORIGIN_BWI',
 'ORIGIN_DCA',
 'ORIGIN_IAD',
 'Flight Status_Delayed',
 'Flight Status_On-time']

In [16]:
# Mode of dummies
print(df1.mode().count())

Binned_CRS_DEP_TIME      1
Weather                  1
DAY_WEEK                 1
CARRIER_CO               1
CARRIER_DH               1
CARRIER_DL               1
CARRIER_MQ               1
CARRIER_OH               1
CARRIER_RU               1
CARRIER_UA               1
CARRIER_US               1
DEST_EWR                 1
DEST_JFK                 1
DEST_LGA                 1
ORIGIN_BWI               1
ORIGIN_DCA               1
ORIGIN_IAD               1
Flight Status_Delayed    1
Flight Status_On-time    1
dtype: int64


In [17]:
# Removing the redundant dummies
# Placeholder variable: rdummies
df2 = df1.copy()
rdummies = ['Flight Status_On-time']
df2 = df1.drop(columns=rdummies)
print(df2.columns.values)

['Binned_CRS_DEP_TIME' 'Weather' 'DAY_WEEK' 'CARRIER_CO' 'CARRIER_DH'
 'CARRIER_DL' 'CARRIER_MQ' 'CARRIER_OH' 'CARRIER_RU' 'CARRIER_UA'
 'CARRIER_US' 'DEST_EWR' 'DEST_JFK' 'DEST_LGA' 'ORIGIN_BWI' 'ORIGIN_DCA'
 'ORIGIN_IAD' 'Flight Status_Delayed']


In [18]:
print(df2.head(5))
print(df2.tail(5))

   Binned_CRS_DEP_TIME  Weather  ...  ORIGIN_IAD  Flight Status_Delayed
0                    5        0  ...           1                      1
1                    3        0  ...           1                      1
2                    8        0  ...           1                      1
3                    4        0  ...           1                      1
4                    5        0  ...           1                      1

[5 rows x 18 columns]
      Binned_CRS_DEP_TIME  Weather  ...  ORIGIN_IAD  Flight Status_Delayed
1314                    1        0  ...           0                      0
1315                    6        0  ...           1                      0
1316                    6        0  ...           0                      0
1317                    4        0  ...           0                      0
1318                    6        0  ...           0                      0

[5 rows x 18 columns]


In [19]:
# Part: Data Partiton
from sklearn.model_selection import train_test_split

# Placeholder variables: df4partition, testpart_size
# test_size specifies the percentage for the test partition
df4partition = df2
testpart_size = 0.2 #Using 20% of the data for testing

# random_state specifies the seed for random number generator. 
# random_state = 1 unless otherwised noted
df_nontestData, df_testData = train_test_split(df4partition, test_size=testpart_size, random_state=1)

print(df_nontestData)

      Binned_CRS_DEP_TIME  Weather  ...  ORIGIN_IAD  Flight Status_Delayed
710                     5        0  ...           0                      0
1258                    4        0  ...           1                      0
435                     7        0  ...           0                      0
987                     6        1  ...           1                      1
1286                    6        0  ...           0                      0
...                   ...      ...  ...         ...                    ...
715                     6        0  ...           1                      0
905                     2        0  ...           1                      0
1096                    8        0  ...           0                      1
235                     5        0  ...           0                      1
1061                    2        0  ...           0                      1

[1055 rows x 18 columns]


In [21]:
# Part Nearest Neighbor 
# Required package: scikit-learn. Package name in Python: sklearn
# Required subpackage: tree 
# Required function name: KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier


# Separate the predictor values and the DV values into X and y respectively
# Placeholder variable: DV
DV = 'Flight Status_Delayed'
X = df_nontestData.drop(columns=[DV])
y = df_nontestData[DV]


In [30]:
# Run Nearest Neighbors Classifier with pre-specified k
# Placeholder variable: k
k = 5
clf = KNeighborsClassifier(metric='euclidean', n_neighbors=k).fit(X,y)

# Report the performance over the test partition
X_test = df_testData.drop(columns=DV)
y_test = df_testData[DV]


from sklearn import metrics

model_object = clf

# Obtain the AUC of the model with k = 5
print(metrics.roc_auc_score(y_test, model_object.predict_proba(X_test)[:, 1]))

0.6020650813516897


In [36]:
# Run Crossvalidation to search for optimal k
# Placeholder variable: kfolds
kfolds = 5

# We first define the search scope. k will be increased from min_k to max_k
max_k = 200
min_k = 1
param_grid = {'n_neighbors': list(range(min_k, max_k+1))}

from sklearn.model_selection import GridSearchCV

gridsearch = GridSearchCV(KNeighborsClassifier(metric='euclidean'),param_grid, scoring='roc_auc', cv=kfolds, n_jobs=-1)
gridsearch.fit(X,y)
clf_bestKNN = gridsearch.best_estimator_

In [59]:
# Display the optimal k that generates the highest AUC
print(clf_bestKNN.n_neighbors)

# Getting the AUC for the optimal kNN model

X_test = df_testData.drop(columns=DV)
y_test_actual = df_testData[DV]

from sklearn.metrics import roc_auc_score

print(roc_auc_score(y_test_actual, clf_bestKNN.predict_proba(X_test)[:,1]))



21
0.6621714643304131
